In [2]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.4.0


In [3]:
# fashion_mnist = keras.datasets.fashion_mnist

# (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

r = np.load("autodata.npz")
r.files
index = np.arange(800)
np.random.shuffle(index)
train_images = r['data'][index][0:700]
train_labels = r['label'][index][0:700]

test_images = r['data'][index][700:]
test_labels = r['label'][index][700:]

print(train_images.shape)
print(test_images.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'autodata.npz'

In [12]:
class_names = ['stand', 'run']

## 浏览数据

在训练模型之前，我们先浏览一下数据集的格式。

In [13]:
train_images.shape

(700, 24, 17, 3)

In [14]:
len(train_labels)

700

每个标签都是一个 0 到 1 之间的整数：

In [15]:
train_labels

array([0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,

测试集中有 100 个图像。同样，每个图像都由 28x28 个像素表示：

In [16]:
test_images.shape

(100, 24, 17, 3)

测试集包含 100 个图像标签：

In [17]:
len(test_labels)

100

## 构建模型

构建神经网络需要先配置模型的层，然后再编译模型。

### 设置层

神经网络的基本组成部分是*层*。层会从向其馈送的数据中提取表示形式。希望这些表示形式有助于解决手头上的问题。

大多数深度学习都包括将简单的层链接在一起。大多数层（如 `tf.keras.layers.Dense`）都具有在训练期间才会学习的参数。

In [1]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(24, 17, 3)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2)
])

NameError: name 'keras' is not defined

该网络的第一层 `tf.keras.layers.Flatten` 将图像格式从二维数组（28 x 28 像素）转换成一维数组（28 x 28 = 784 像素）。将该层视为图像中未堆叠的像素行并将其排列起来。该层没有要学习的参数，它只会重新格式化数据。

展平像素后，网络会包括两个 `tf.keras.layers.Dense` 层的序列。它们是密集连接或全连接神经层。第一个 `Dense` 层有 128 个节点（或神经元）。第二个（也是最后一个）层会返回一个长度为 10 的 logits 数组。每个节点都包含一个得分，用来表示当前图像属于 10 个类中的哪一类。

### 编译模型

在准备对模型进行训练之前，还需要再对其进行一些设置。以下内容是在模型的*编译*步骤中添加的：

- *损失函数* - 用于测量模型在训练期间的准确率。您会希望最小化此函数，以便将模型“引导”到正确的方向上。
- *优化器* - 决定模型如何根据其看到的数据和自身的损失函数进行更新。
- *指标* - 用于监控训练和测试步骤。以下示例使用了*准确率*，即被正确分类的图像的比率。

In [42]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## 训练模型

训练神经网络模型需要执行以下步骤：

1. 将训练数据馈送给模型。在本例中，训练数据位于 `train_images` 和 `train_labels` 数组中。
2. 模型学习将图像和标签关联起来。
3. 要求模型对测试集（在本例中为 `test_images` 数组）进行预测。
4. 验证预测是否与 `test_labels` 数组中的标签相匹配。


### 向模型馈送数据

要开始训练，请调用 `model.fit` 方法，这样命名是因为该方法会将模型与训练数据进行“拟合”：

In [59]:
model.fit(train_images, train_labels, epochs=10)

Epoch 1/10
22/22 [==============================] - 0s 2ms/step - loss: 0.1757 - accuracy: 0.9414
Epoch 2/10
22/22 [==============================] - 0s 2ms/step - loss: 0.1662 - accuracy: 0.9414
Epoch 3/10
22/22 [==============================] - 0s 2ms/step - loss: 0.1618 - accuracy: 0.9443
Epoch 4/10
22/22 [==============================] - 0s 2ms/step - loss: 0.1627 - accuracy: 0.9529
Epoch 5/10
22/22 [==============================] - 0s 2ms/step - loss: 0.2728 - accuracy: 0.8871
Epoch 6/10
22/22 [==============================] - 0s 2ms/step - loss: 0.2483 - accuracy: 0.8957
Epoch 7/10
22/22 [==============================] - 0s 2ms/step - loss: 0.1488 - accuracy: 0.9543
Epoch 8/10
22/22 [==============================] - 0s 2ms/step - loss: 0.1606 - accuracy: 0.9471
Epoch 9/10
22/22 [==============================] - 0s 2ms/step - loss: 0.1911 - accuracy: 0.9343
Epoch 10/10
22/22 [==============================] - 0s 2ms/step - loss: 0.3676 - accuracy: 0.8314


在模型训练期间，会显示损失和准确率指标。此模型在训练数据上的准确率达到了 0.91（或 91%）左右。

### 评估准确率

接下来，比较模型在测试数据集上的表现：

In [60]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

4/4 - 0s - loss: 0.4705 - accuracy: 0.7500

Test accuracy: 0.75


结果表明，模型在测试数据集上的准确率略低于训练数据集。训练准确率和测试准确率之间的差距代表*过拟合*。过拟合是指机器学习模型在新的、以前未曾见过的输入上的表现不如在训练数据上的表现。过拟合的模型会“记住”训练数据集中的噪声和细节，从而对模型在新数据上的表现产生负面影响。有关更多信息，请参阅以下内容：

- [演示过拟合](https://tensorflow.google.cn/tutorials/keras/overfit_and_underfit#demonstrate_overfitting)
- [避免过拟合的策略](https://tensorflow.google.cn/tutorials/keras/overfit_and_underfit#strategies_to_prevent_overfitting)

### 进行预测

在模型经过训练后，您可以使用它对一些图像进行预测。模型具有线性输出，即 [logits](https://developers.google.com/machine-learning/glossary#logits)。您可以附加一个 softmax 层，将 logits 转换成更容易理解的概率。 

In [30]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [64]:
predictions = probability_model.predict(test_images)

在上例中，模型预测了测试集中每个图像的标签。我们来看看第一个预测结果：

In [65]:
predictions[0]

array([0.07343584, 0.92656416], dtype=float32)

预测结果是一个包含 10 个数字的数组。它们代表模型对 10 种不同服装中每种服装的“置信度”。您可以看到哪个标签的置信度值最大：

In [33]:
np.argmax(predictions[0])

1

因此，该模型非常确信这个图像是短靴，或 `class_names[9]`。通过检查测试标签发现这个分类是正确的：

In [34]:
test_labels[0]

1

## 使用训练好的模型

最后，使用训练好的模型对单个图像进行预测。

In [35]:
# Grab an image from the test dataset.
img = test_images[1]

print(img.shape)

(24, 17, 3)


`tf.keras` 模型经过了优化，可同时对一个*批*或一组样本进行预测。因此，即便您只使用一个图像，您也需要将其添加到列表中：

In [36]:
# Add the image to a batch where it's the only member.
img = (np.expand_dims(img,0))

print(img.shape)

(1, 24, 17, 3)


现在预测这个图像的正确标签：

In [37]:
predictions_single = probability_model.predict(img)

print(predictions_single)

[[0.99889034 0.00110969]]


`keras.Model.predict` 会返回一组列表，每个列表对应一批数据中的每个图像。在批次中获取对我们（唯一）图像的预测：

In [38]:
np.argmax(predictions_single[0])

0

## 最后保存模型

In [66]:
probability_model.save('model.h5')
model.save('premodel.h5')